In [ ]:
!nvidia-smi

Tue Sep 19 09:41:00 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')



# !pip install tensorflow
# !pip install keras

import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator, load_img
from glob import glob

from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout
from keras.applications import  VGG16
from keras.applications import ResNet50
from keras.applications import InceptionV3

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ROOT_PATH = '/content/drive/MyDrive'

In [ ]:
!pwd

/content


In [ ]:

import os
os.chdir(ROOT_PATH)
os.getcwd()

'/content/drive/MyDrive'

In [ ]:
!ls


 12certificate.pdf
 4th_Sem_ResearchPapers
'Activity Diagram'
'Activity .pdf'
'B3_G13_WBS - WBS.gsheet'
'Colab Notebooks'
'Copy of Linkedin_screenshot.jpg'
 Diabetic_Retinopathy_Dataset
 Diabetic_Retinopathy_Dataset_old
'Document from Maaz Sayyed'
'Document from Maaz Sayyed (1)'
'Document from Maaz Sayyed (1).pdf'
'Document from Maaz Sayyed (2).pdf'
'Document from Maaz Sayyed.pdf'
 ds
 DSAAT.pdf
'DS_Block chain and PageRank Algorithm.pdf'
'ds lab'
'DS_Literature  Review (1).gdoc'
'DS_Literature  Review.gdoc'
' HEALTHY MINDS  LOGIC.gdoc'
 hsc_marksheet.txt
 Images
 Images_Arts
 IT_A_12110133.pdf
 IT_A_12110133_VATSALYA
 Kaagaz_20220320_1917506242.pdf
 Linkedin_screenshot.jpg
'MAR Lab Journal Index page and Front page June 22.pdf'
'Muaz Sayyed_49_Linked List'
'Muaz Sayyed_49_Linked List (1)'
 Narasimha_Karumanchi_Data_Structures.gdoc
'P_49_MAD LAB_1.docx'
'Photo from Maaz Sayyed'
'Resume (1).gdoc'
'Resume (2).gdoc'
 Resume.gdoc
'Simio Assignment 3.spfx'
 SystemDesign.gdoc
'Untitled docum

In [ ]:
train_path      =  os.path.join('/content/drive/MyDrive/Diabetic_Retinopathy_Dataset', 'train')

validation_path =  os.path.join('/content/drive/MyDrive/Diabetic_Retinopathy_Dataset', 'validation')

test_path       =  os.path.join('/content/drive/MyDrive/Diabetic_Retinopathy_Dataset', 'test')

In [ ]:
train_data_gen = ImageDataGenerator(
                rescale=1./255,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True
)


validation_data_gen = ImageDataGenerator(
                rescale=1./255,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True
)


test_data_gen = ImageDataGenerator(rescale=1./255)


            # logging.info("Image Data Generator Initiated Successfully.")

In [ ]:
train_data_set = train_data_gen.flow_from_directory(
                train_path,
                target_size=(224,224),
                batch_size=32,
                class_mode='categorical',
                color_mode='rgb',
                shuffle=True
)

validation_data_set = validation_data_gen.flow_from_directory(
                validation_path,
                target_size=(224,224),
                batch_size=32,
                class_mode='categorical',
                color_mode='rgb',
                shuffle=True
)

test_data_set = test_data_gen.flow_from_directory(
                test_path,
                target_size=(224,224),
                batch_size=32,
                class_mode='categorical',
                color_mode='rgb',
                shuffle=True
)

Found 2744 images belonging to 5 classes.
Found 369 images belonging to 5 classes.
Found 549 images belonging to 5 classes.


In [ ]:
def model_compile_fit(base_models,train_data_set,validation_data_set,test_data_set):

    accuracy_score_list = []
    loss_score_list = []
    model_fit_list = []
    model_history_list = []

    num_output_class = len(train_data_set.class_indices)

    for base_model in list(base_models.values()):
        model = Sequential()
        model.add(base_model)

        # Add Fully Connected Layers
        model.add(Flatten())
        model.add(Dense(units=256,activation='relu'))
        model.add(Dropout(0.4))
        model.add(Dense(units=num_output_class,activation='softmax'))

        # Compile the Model
        model.compile(
            loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy']
        )

        model.fit(
            train_data_set,
            validation_data=validation_data_set,
            epochs=7,
            steps_per_epoch=len(train_data_set),
            validation_steps= len(validation_data_set)
        )

        model_fit_list.append(model)
        model_history.append(model.history)

        # logging.info(Add a dataframe to show each model evaluation using pd.DataFrame(model.history) [loss, acc, val_lss, val_acc])



        # Make Predictions
        # predictions = model.predict(test_data_set)

        # Calculate Accuracy and Loss
        # true_labels = test_data_set.classes
        # predicted_labels = np.argmax(predictions, axis=1)


        # Calculate accuracy(Since we have not used One-Hot Encoder)
        # If you're using categorical cross-entropy loss during training

        loss, accuracy = model.evaluate(test_data_set)

        # print(f'Loss: {loss[0]}')
        # accuracy = np.sum(true_labels == predicted_labels) / len(true_labels)
        # print(f'Accuracy: {accuracy}')

        accuracy_score_list.append(accuracy)
        loss_score_list.append(loss)

    # ------------------------------------------------

        # To determine which class is given which label
        # class_indices = train_generator.class_indices
        # class_indices


        # class_mapping = {
        #     0: 'type1',
        #     1: 'type2',
        #     2: 'type3',
        #     3: 'type4',
        #     4: 'type5'
        # }

        # # Assuming 'predicted_labels' is the output of your model
        # predicted_class_names = [class_mapping[label] for label in predicted_labels]

    # -----------------------------------------------------



    return (
        accuracy_score_list,
        loss_score_list,
        model_fit_list,
        model_history_list
    )




In [ ]:
IMAGE_SIZE = [224,224]
CHANNELS = [3]  # RGB


            # Model Dict.
            # Define model fit, compile, define base_model  in utils

base_models = {
                'VGG16'           : VGG16(input_shape=IMAGE_SIZE+CHANNELS, weights='imagenet', include_top=False),
                'RESNET50'        : ResNet50(weights='imagenet', include_top=False, input_shape=IMAGE_SIZE+CHANNELS),
                'InceptionV3'     : InceptionV3(input_shape=IMAGE_SIZE+CHANNELS, weights='imagenet', include_top=False),
            }


accuracy_score_list, loss_score_list, model_fit_list, model_history_list = model_compile_fit(base_models,train_data_set,validation_data_set,test_data_set)

best_score_index = accuracy_score_list.index(max(accuracy_score_list))

best_model_name = list(base_models.keys())[best_score_index]

best_model = model_fit_list[best_score_index]

best_model_loss = loss_score_list[best_score_index]

best_model_accuracy = accuracy_score_list[best_score_index]

best_model_history = model_history_list[best_score_index]


87910968/87910968 [==============================] - 1s 0us/step
Epoch 1/7
86/86 [==============================] - 627s 7s/step - loss: 1.4111 - accuracy: 0.4894 - val_loss: 0.9645 - val_accuracy: 0.6775
Epoch 2/7
86/86 [==============================] - 58s 678ms/step - loss: 1.0779 - accuracy: 0.6235 - val_loss: 0.9368 - val_accuracy: 0.6612
Epoch 3/7
86/86 [==============================] - 59s 683ms/step - loss: 0.9660 - accuracy: 0.6669 - val_loss: 0.9630 - val_accuracy: 0.6667
Epoch 4/7
86/86 [==============================] - 59s 683ms/step - loss: 0.9347 - accuracy: 0.6760 - val_loss: 0.8866 - val_accuracy: 0.6802
Epoch 5/7
86/86 [==============================] - 58s 675ms/step - loss: 0.8925 - accuracy: 0.6888 - val_loss: 0.7971 - val_accuracy: 0.7182
Epoch 6/7
86/86 [==============================] - 59s 685ms/step - loss: 0.8998 - accuracy: 0.6877 - val_loss: 0.8158 - val_accuracy: 0.6938
Epoch 7/7
18/18 [==============================] - 98s 6s/step - loss: 0.7801 - accur

In [ ]:
base_models

{'VGG16': <keras.src.engine.functional.Functional at 0x79bbba941480>,
 'RESNET50': <keras.src.engine.functional.Functional at 0x79bbb80d82b0>,
 'InceptionV3': <keras.src.engine.functional.Functional at 0x79bba025f6a0>}

In [ ]:
accuracy_score_list

[0.7231329679489136, 0.2732240557670593, 0.750455379486084]

In [ ]:
loss_score_list

[0.7800658345222473, 1.5271400213241577, 0.6774214506149292]

In [ ]:
model_fit_list

In [ ]:
best_score_index = accuracy_score_list.index(max(accuracy_score_list))

best_model_name = list(base_models.keys())[best_score_index]

best_model = model_fit_list[best_score_index]

best_model_loss = loss_score_list[best_score_index]

best_model_accuracy = accuracy_score_list[best_score_index]

In [ ]:
best_score_index

2

In [ ]:
best_model_accuracy

0.750455379486084

In [ ]:
best_model_loss

0.6774214506149292

In [ ]:
best_model_name

'InceptionV3'

In [ ]:
best_model

In [ ]:
scores = best_model.evaluate(test_data_set)

18/18 [==============================] - 3s 151ms/step - loss: 0.6774 - accuracy: 0.7505


In [ ]:
scores

[0.6774214506149292, 0.750455379486084]

In [ ]:

        # Make Predictions
predictions = best_model.predict(test_data_set)

# Calculate Accuracy and Loss
true_labels = test_data_set.classes
predicted_labels = np.argmax(predictions, axis=1)

18/18 [==============================] - 4s 154ms/step


In [ ]:
true_labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [ ]:
predicted_labels

array([1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 2,
       1, 2, 1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 2, 1, 1,
       1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1,
       2, 1, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2,
       1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1,
       2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2,
       1, 2, 1, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 1, 2, 1,
       1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 2, 1, 1, 1, 2, 1, 1, 2, 1,
       2, 1, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 1, 2, 1, 2, 1, 1, 2,
       2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1,
       2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 1, 1, 2, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2, 2, 1, 2, 1, 1, 2, 1, 2, 1,
       2, 1, 1, 1, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 2, 1,
       2, 1, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 1, 1, 2,

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, f1_score

In [ ]:
c_mat = confusion_matrix(y_true=true_labels,y_pred=predicted_labels)

In [ ]:
c_mat

array([[  0,  30,  25,   0,   0],
       [  0,  72,  78,   0,   0],
       [  0, 132, 139,   0,   0],
       [  0,  24,  20,   0,   0],
       [  0,  16,  13,   0,   0]])

In [ ]:
accuracy_score(y_true=true_labels,y_pred=predicted_labels)

0.3843351548269581

In [ ]:
class_labels = list(test_data_set.class_indices.keys())
class_labels

['Mild', 'Moderate', 'No_DR', 'Proliferate_DR', 'Severe']

In [ ]:
train_data_set.class_indices

{'Mild': 0, 'Moderate': 1, 'No_DR': 2, 'Proliferate_DR': 3, 'Severe': 4}

In [ ]:
accuracy = np.mean(predicted_labels == true_labels)
accuracy

0.3843351548269581

In [ ]:
best_model.history